# Neo4j Hello World (Notebook)

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it into a pandas DataFrame.

**Assumes** Neo4j is running at `bolt://localhost:${URI_PORT}` with the user and password set in the `.env` file.


In [107]:
# Optional: install dependencies if running outside the repo venv
# !pip install neo4j pandas
import os
from dotenv import load_dotenv  
load_dotenv()  # Load local environment variables

URI = "bolt://localhost:" + os.environ.get("URI_PORT")
NEO4J_USER = os.environ.get("NEO4J_USER")
NEO4J_PWD = os.environ.get("NEO4J_PASSWORD")
print(f"Connecting to Neo4j at {URI} with user {NEO4J_USER} and password {NEO4J_PWD}")

Connecting to Neo4j at bolt://localhost:7687 with user neo4j and password test1234


In [108]:
import yaml
from pathlib import Path

# load cypher queries from yaml file
queries = yaml.safe_load(Path("queries.yaml").read_text())
queries.keys()  # list available queries

dict_keys(['create_seed', 'match_person', 'match_company', 'match_adjacency', 'delete_all'])

## Using the Neo4j driver

In [109]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(URI, auth=(NEO4J_USER, NEO4J_PWD))
driver

In [110]:
wipe_at_init = True  # set True to delete everything at the start
wipe_at_end = False  # set True to delete everything at the end

with driver.session() as session:
    
    print("\n== Init Cleanup.")
    if wipe_at_init:
        session.run(queries["delete_all"])
        print("ok")
    else:
        print("skipped")
        
    print("\n== Creating sample data")
    
    session.run(queries["create_seed"])
    print("ok")

    print("\n== Query: list all people")
    records = session.run(queries["match_person"]) # <class 'neo4j.work.result.Result'>, convert to  <class 'list'> to see contents: list(records)
    for r in records:
        print(r)
        
    
    print("\n== Query: list all companies")
    records = session.run(queries["match_company"]) # <class 'neo4j.work.result.Result'>, convert to  <class 'list'> to see contents: list(records)
    for r in records:
        print(r)

    print("\n== Query: adjacency (who knows whom)")
    records = session.run(queries["match_adjacency"]) # <class 'neo4j.work.result.Result'>, convert to  <class 'list'> to see contents: list(records)
    for r in records:
        print(r)
        
    print("\n== End Cleanup.")
    if wipe_at_end:
        session.run(queries["delete_all"])
        print("ok")
    else:
        print("skipped")


== Init Cleanup.
ok

== Creating sample data
ok

== Query: list all people
<Record name='Paula' age=25 education='Computer Engineering'>
<Record name='Guillermo' age=26 education='Industrial Engineering'>
<Record name='Gabriela' age=26 education='Physics'>
<Record name='Iria' age=27 education='Physics'>
<Record name='Cristina' age=27 education='Physics'>

== Query: list all companies
<Record company_name='Indra' industry='Engineering'>
<Record company_name='CIEMAT' industry='Scientific Research'>
<Record company_name='CBM' industry='Scientific Research'>

== Query: adjacency (who knows whom)
<Record person='Cristina' knows=['Gabriela', 'Iria'] works_at='CBM'>
<Record person='Gabriela' knows=['Cristina', 'Iria'] works_at='CIEMAT'>
<Record person='Guillermo' knows=['Paula', 'Iria'] works_at='Indra'>
<Record person='Iria' knows=['Cristina', 'Paula', 'Gabriela', 'Guillermo'] works_at='Indra'>
<Record person='Paula' knows=['Guillermo', 'Iria'] works_at='Indra'>

== End Cleanup.
skipped


In [111]:
driver.close()
print("Driver closed.")

Driver closed.


## Using Langchain wrapper for Neo4j

In [112]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=URI, username=NEO4J_USER, password=NEO4J_PWD)
print("\n== Clearing database (DETACH DELETE all nodes)")
wipe_at_init = True  # set True to delete everything at the start
wipe_at_end = False  # set True to delete everything at the end

print("\n== Init Clean up.")
if wipe_at_init:
    graph.query(queries["delete_all"])
    print("ok")
else:
    print("skipped")
    
print("\n== Creating sample data")
graph.query(queries["create_seed"])
print("ok")

print("\n== Query: list all people")
records = graph.query(queries["match_person"]) # <class 'list'>
for r in records:
    print(r)
    
print("\n== Query: list all companies")
records = graph.query(queries["match_company"]) # <class 'list'>
for r in records:
    print(r)

print("\n== Query: adjacency (who knows whom)")
records = graph.query(queries["match_adjacency"]) # <class 'list'>
for r in records:
    print(r)

print("\n== End Cleanup.")
if wipe_at_end:
    graph.query(queries["delete_all"])
    print("ok")
else:
    print("skipped")
    
print("\n== Done.")



== Clearing database (DETACH DELETE all nodes)

== Init Clean up.
ok

== Creating sample data
ok

== Query: list all people
{'name': 'Paula', 'age': 25, 'education': 'Computer Engineering'}
{'name': 'Guillermo', 'age': 26, 'education': 'Industrial Engineering'}
{'name': 'Gabriela', 'age': 26, 'education': 'Physics'}
{'name': 'Iria', 'age': 27, 'education': 'Physics'}
{'name': 'Cristina', 'age': 27, 'education': 'Physics'}

== Query: list all companies
{'company_name': 'Indra', 'industry': 'Engineering'}
{'company_name': 'CIEMAT', 'industry': 'Scientific Research'}
{'company_name': 'CBM', 'industry': 'Scientific Research'}

== Query: adjacency (who knows whom)
{'person': 'Cristina', 'knows': ['Gabriela', 'Iria'], 'works_at': 'CBM'}
{'person': 'Gabriela', 'knows': ['Cristina', 'Iria'], 'works_at': 'CIEMAT'}
{'person': 'Guillermo', 'knows': ['Paula', 'Iria'], 'works_at': 'Indra'}
{'person': 'Iria', 'knows': ['Cristina', 'Paula', 'Gabriela', 'Guillermo'], 'works_at': 'Indra'}
{'person': '